# Elasticsearchを利用したサンプル検索

Elasticsearchを利用した検索などを試すためのノートブックです。

## 接続先の設定

**ESHOST**に接続先を設定します。サンプルでは、ローカルネットワーク環境にあるElasticsearchに接続しています。
接続先の各種設定については公式ドキュメントをご覧ください。
https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html

**INDEX**が接続先のインデックス名です。

In [6]:
from elasticsearch import Elasticsearch
import json

ESHOST = "http://es:9200"
INDEX = "esci-products"

esclient = Elasticsearch(ESHOST)

## スキーマの確認

対象となるインデックスのスキーマを確認します。

In [7]:
import json
response = esclient.indices.get(index=INDEX, pretty=True)
print(json.dumps(response.body, indent=2))

{
  "esci-products": {
    "aliases": {},
    "mappings": {
      "properties": {
        "image": {
          "type": "keyword",
          "index": false,
          "doc_values": false
        },
        "product_brand": {
          "type": "keyword"
        },
        "product_bullet_point": {
          "type": "keyword",
          "ignore_above": 256,
          "fields": {
            "en": {
              "type": "text",
              "analyzer": "standard"
            },
            "ja": {
              "type": "text",
              "analyzer": "kuromoji"
            }
          }
        },
        "product_color": {
          "type": "keyword"
        },
        "product_description": {
          "type": "keyword",
          "ignore_above": 256,
          "fields": {
            "en": {
              "type": "text",
              "analyzer": "standard"
            },
            "ja": {
              "type": "text",
              "analyzer": "kuromoji"
            }
          }

## データの確認

件数を確認する

In [8]:
response = esclient.count(
    index=INDEX
)
response

ObjectApiResponse({'count': 339059, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})


検索してみる。

In [9]:
import pandas as pd

response = esclient.search(
    index=INDEX
)

result_df = pd.DataFrame(d["_source"] for d in response["hits"]["hits"])
result_df

,product_id,product_title,product_description,product_bullet_point,product_brand,product_color,product_locale,image,type,products_dense_vector
0,0007350775,Pride and Prejudice (Collins Classics),None,None,None,None,jp,None,book,"[0.4097869098186493, -2.0519256591796875, -1.4..."
1,0007420277,The Phantom of the Opera (Collins Classics),None,None,None,None,jp,None,book,"[0.4534071683883667, 0.7054824233055115, 2.556..."
2,000843171X,i-SPY Garden Birds: Spy it! Score it! (Collins...,None,None,None,None,jp,None,book,"[-1.469808578491211, 0.289233922958374, 1.2351..."
3,0008431728,i-SPY Wildflowers: Spy it! Score it! (Collins ...,None,None,None,None,jp,NaN,NaN,"[-1.9517138004302979, 1.2041120529174805, 1.72..."
4,0008431752,i-SPY Trees: Spy it! Score it! (Collins Michel...,None,None,None,None,jp,None,book,"[-3.360761880874634, -0.3101186752319336, 0.86..."
5,0008431795,i-SPY Butterflies and Moths: Spy it! Score it!...,None,None,None,None,jp,None,error,"[-2.095010757446289, -0.48692020773887634, -0...."
6,0008431809,i-SPY On a Car Journey in France: Spy it! Scor...,None,None,None,None,jp,None,book,"[-2.7170090675354004, 0.7793845534324646, -0.5..."
7,0008431817,i-SPY Cool Cars: Spy it! Score it! (Collins Mi...,None,None,None,None,jp,None,book,"[-1.4191445112228394, 0.9483093023300171, -0.9..."
8,0008431825,i-SPY London: Spy it! Score it! (Collins Miche...,None,None,None,None,jp,None,book,"[-2.986598491668701, -0.09516282379627228, 0.5..."
9,0046295712,横川晴児 : 音階と運指 (クラリネット教則本) ルデュック出版,None,原題 : YOKOKAWA : LES GAMMES ET LES DOIGTES SOUS...,ウィットナー,None,jp,https://m.media-amazon.com/images/I/51DIIfeOKh...,product,"[0.21565665304660797, 0.36088088154792786, -0...."


## クエリを指定して検索


In [12]:
query = {
    "match": {
        "product_color": "ブラック"
    }
}

response = esclient.search(
    index=INDEX,
    query=query,
    track_total_hits=True
)

print(f"{response['hits']['total']['value']}件ヒット")
result_df = pd.DataFrame(d["_source"] for d in response["hits"]["hits"])
result_df

{'value': 23893, 'relation': 'eq'}件ヒット


,product_id,product_title,product_description,product_bullet_point,product_brand,product_color,product_locale,image,type,products_dense_vector
0,8883701038,"Caderno Clássico, Preto, Capa Dura, Sem Pauta,...",None,192 PÁGINAS\nCAPA DURA\nPAPEL MARFIM ACID-FREE...,Moleskine(モレスキン),ブラック,jp,None,book,"[0.9062901735305786, 0.8304082155227661, 1.462..."
1,B00002N6SL,MAG-LITE(マグライト) 2AA BP BK M2A016V,None,燭光2200cp\nランプ寿命3~4回(電池を交換する回数)\n電池寿命/連続点灯6~7時間...,MAG-LITE(マグライト),ブラック,jp,https://m.media-amazon.com/images/I/81qbd9XyLJ...,product,"[-0.5576668977737427, 0.765504002571106, 1.350..."
2,B00004OCKO,OXO 軽量 柔らかグリップ ワインオープナー,None,<b>メーカー型番:</b> 31781\n<b>サイズ:</b> 幅7.5×長さ20×厚み...,OXO (オクソー),ブラック,jp,https://m.media-amazon.com/images/I/711CfjjgrL...,product,"[-0.7368791103363037, 1.1023060083389282, -1.8..."
3,B00004SD7B,FISKARS (フィスカース) Fiskars/フィスカース 7854 X25 ガーデンア...,None,28-Inch,FISKARS (フィスカース),ブラック,jp,https://m.media-amazon.com/images/I/419bsJdbXu...,product,"[-2.5628771781921387, 0.9464580416679382, 0.14..."
4,B0000536M2,MAYBELLINE FULL N' SOFT MASCARA THICK & HEALTH...,MASCARA | FULL N' SOFT MASCARA THICK & HEALTHY,MASCARA | FULL N' SOFT MASCARA THICK & HEALTHY,MAYBELLINE,ブラック,jp,https://m.media-amazon.com/images/I/71SG3xJHMN...,product,"[-0.44955873489379883, 0.3860465884208679, -1...."
5,B000068O1A,Hosa CPP-201 1m 両側モノラルフォンプラグ×2 オーディオケーブル,None,ホサ\n世界的なケーブルメーカー『Hosa（ホサ）』のオーディオケーブルです。\n【特長】・...,Hosa,ブラック,jp,https://m.media-amazon.com/images/I/814XTim7ND...,product,"[-1.0535533428192139, 0.3041990399360657, 1.15..."
6,B000068O1N,Hosa STP-201 1m ステレオフォンオス-モノラルフォンオス×2 インサーションケーブル,None,ホサ\nコネクタにステレオフォンオス（1/4” TRS）とモノラルフォンオス（1/4” TS...,Hosa,ブラック,jp,https://m.media-amazon.com/images/I/31rkpsoWgn...,product,"[-1.2108471393585205, 0.35665908455848694, 0.3..."
7,B000068O3D,Hosa CMP-110 3m ステレオミニプラグ-モノラルフォンプラグ オーディオケーブル,None,ホサ\n世界的なケーブルメーカー『Hosa（ホサ）』のオーディオケーブルです。\nステレオミ...,Hosa,ブラック,jp,https://m.media-amazon.com/images/I/21tvFi8u1J...,product,"[-0.39471518993377686, 0.11549869179725647, 0...."
8,B000068O3F,Hosa CMP-105 1.5m ステレオミニプラグ-モノラルフォンプラグ オーディオケーブル,None,ホサ\n世界的なケーブルメーカー『Hosa（ホサ）』のオーディオケーブルです。\nステレオミ...,Hosa,ブラック,jp,https://m.media-amazon.com/images/I/71eTICqQu4...,product,"[-0.4096856713294983, 0.12733402848243713, 0.3..."
9,B00006JSUF,LODGE(ロッジ) ロジック10.25インチキッチンダッチオーブン L8DO3 (並行輸入品),None,None,ロッジ(Lodge),ブラック,jp,https://m.media-amazon.com/images/I/51iSDLDmt2...,product,"[-3.032054901123047, 0.5627821683883667, 0.965..."


## ベクトル検索（knn検索）

`text`の文字列をLLMのモデルを使って、ベクトルに変換し、knn検索を実行する。
インデックスにはすでに同じLLMのモデルを使用してドキュメントごとにベクトルが付与されているものとする。


In [ ]:
import torch
import japanese_clip as ja_clip

text = "黒のリュックカバー"

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"device is {device}")
model, preprocess = ja_clip.load("rinna/japanese-clip-vit-b-16", cache_dir="/tmp/japanese_clip", device=device)
tokenizer = ja_clip.load_tokenizer()
encodings = ja_clip.tokenize(
    texts=text,
    device=device,
    tokenizer=tokenizer, # this is optional. if you don't pass, load tokenizer each time
)

with torch.no_grad():
    embeddings = model.get_text_features(**encodings)[0].tolist()
    

knn_query = {
    "field": "products_dense_vector",
    "query_vector": embeddings,
    "k": 10,
    "num_candidates": 100
}

response = esclient.search(
    index=INDEX,
    knn=knn_query
)

result_df = pd.DataFrame(d["_source"] for d in response["hits"]["hits"])
result_df